Essential Libraries

In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 8
CHANNELS = 3
EPOCHS = 25
DIR_NAME = "datasets/PlantVillage/Split/" 
N_CLASSES = 10

TRAIN_DIR = DIR_NAME + "train/"
VAL_DIR = DIR_NAME + "val/"
TEST_DIR = DIR_NAME + "test/"

In [3]:
# create a data generator
datagen = ImageDataGenerator()

train_it = datagen.flow_from_directory(TRAIN_DIR, batch_size=799)

val_it = datagen.flow_from_directory(VAL_DIR, batch_size=99)

test_it = datagen.flow_from_directory(TEST_DIR, batch_size=99)

Found 800 images belonging to 10 classes.
Found 100 images belonging to 10 classes.
Found 100 images belonging to 10 classes.


In [4]:
X_train, y_train = train_it.next()
X_test, y_test = test_it.next()
X_val, y_val = val_it.next()

In [5]:
idx = (train_it.batch_index - 1) * train_it.batch_size
fname = train_it.filenames[idx : idx + train_it.batch_size]

In [6]:
weights = None
with open('weights.csv', 'r') as file:
    weights = dict(line.strip().split(',') for line in file.readlines())

In [7]:
sample_weights = []
for f in fname:
  basename = f.split('\\')[-1]
  if basename in weights:
    sample_weights.append(float(weights[basename]))
  else:
    sample_weights.append(0)

In [8]:
print(len(sample_weights))

799


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train, sample_weights)).batch(1)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(1)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(1)

In [10]:
# class_names = dataset.class_names
# print(class_names)

In [11]:
# train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset, 0.8, 0.1, 0.1, False)
print("Training Size = ", len(train_dataset))
print("Validation Size = ", len(val_dataset))
print("Testing Size = ", len(test_dataset))

Training Size =  799
Validation Size =  99
Testing Size =  99


In [12]:
# cache the images, using tf prefetch pipeline
train_ds = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [14]:
# TF data argumentation: fix contrast, rotation of images by transformation
data_argumentaion = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

In [15]:
# Convolutional layer = Conv2D
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

model = Sequential([
    resize_and_rescale,
    data_argumentaion,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(N_CLASSES, activation='softmax'),
])

model.build(input_shape=input_shape)

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (8, 256, 256, 3)          0         
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                  

In [17]:
model.compile(
    optimizer='adam',
    # loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    loss='binary_crossentropy',
    metrics=['accuracy']
    # run_eagerly=True
)

In [18]:
# history = model.fit(
#     train_ds,
#     epochs=EPOCHS,
#     verbose=1,
#     validation_data=val_ds
# )
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    verbose=1,
    validation_data=val_ds,
    batch_size=BATCH_SIZE,
)

Epoch 1/25
799/799 [==============================] - 233s 286ms/step - loss: 3.4832e-04 - accuracy: 0.1076 - val_loss: 0.3334 - val_accuracy: 0.1010
Epoch 2/25
799/799 [==============================] - 248s 310ms/step - loss: 3.2351e-04 - accuracy: 0.1464 - val_loss: 0.3221 - val_accuracy: 0.2121
Epoch 3/25
799/799 [==============================] - 262s 328ms/step - loss: 2.9777e-04 - accuracy: 0.2466 - val_loss: 0.2842 - val_accuracy: 0.2828
Epoch 4/25
799/799 [==============================] - 229s 287ms/step - loss: 2.7532e-04 - accuracy: 0.3292 - val_loss: 0.2630 - val_accuracy: 0.3636
Epoch 5/25
799/799 [==============================] - 249s 312ms/step - loss: 2.6180e-04 - accuracy: 0.3830 - val_loss: 0.2669 - val_accuracy: 0.3838
Epoch 6/25
799/799 [==============================] - 248s 310ms/step - loss: 2.5643e-04 - accuracy: 0.4155 - val_loss: 0.2631 - val_accuracy: 0.3535
Epoch 7/25
799/799 [==============================] - 241s 301ms/step - loss: 2.3777e-04 - accuracy:

In [ ]:
score = model.evaluate(test_ds)

In [ ]:
# history.params
# history.history['accuracy']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc, label='Training Accuracy')
plt.plot(range(EPOCHS), val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss, label='Training Loss')
plt.plot(range(EPOCHS), val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

In [ ]:
# # Make Predictions
# '''
# for images_batch, labels_batch in test_ds.take(1):
#     first_image = images_batch[0].numpy().astype('uint8')
#     first_label = labels_batch[0]
    
#     print("First image to predict")
#     plt.imshow(first_image)
#     print("actual label:", class_names[first_label])
    
#     batch_prediction = model.predict(images_batch)
#     predicted_class_index = np.argmax(batch_prediction[0])
#     print("Predicted label: ", class_names[predicted_class_index])
# '''

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0)  #creating a batch
    
    predictions = model.predict(img_array)
    
    predicted_class =  class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])),2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(14,14))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        plt.axis('off')
        
        predicted_class, confidence = predict(model, images[i].numpy())
        
        actual_class = class_names[labels[i]]
        
        plt.title(f"Actual: {actual_class},\nPredicted: {predicted_class}.\nConfidence: {confidence}%")
        

In [ ]:
# save model
model_version=1
model.save(f"../models/{model_version}",save_format='h5')
# model.save("../models/model1")